## Geodata Mask Module Sample Scripts - Test

#### Jeffrey Feng (j1feng@ucsd.edu), July 2021

More details on this notebook can be found in `geodata/doc/jupyter_notebooks`

# Introduction

Geodata is also able to process raster files and geospatial shapefiles. Built off the Rasterio library, its mask module function includes adding shapefiles into binary mask layers, merging and flattening multiple raster images together, and extract region data from merged mask and shapefiles.

Its current functionalities in details are:

- Creating mask, adding layers from .tif files

- CRS convertion, cropping, trimming, binarizing layers

- Merging and flattening layers

- Adding .shp files as layers

- Extracting shapes from merged mask

In [ ]:
import geodata
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
show = geodata.Mask.show
import cartopy.io.shapereader as shpreader

Since this module process raster files, we would use the four following files:
    
- Elevation_Slope.tif
- FINAL_GRID_5BINS.tif
- FINAL_GRID_FOREST_MED.tif
- MODIS_China.tif

These tif files can be downloaded in https://drive.google.com/drive/u/1/folders/1uRnVkzZdY2SU3pzU8TiygBUVBMhxrvaJ

In [ ]:
#These can be relative path if you downloaded these files in the same folder as this notebook
slope_path = ... 
grid_bins_path = ...
forest_med_path = ... 
modis_china_path = ... 

## CREATING OBJECT, ADDING LAYERS

In [ ]:
china = geodata.Mask("China", layer_path = {
    'bins': grid_bins_path, 
    'forest': forest_med_path,
    'slope': slope_path})
china

In [ ]:
china.get_res()

In [ ]:
china.get_bounds()

### CRS convertion, trimming, cropping (if necessary)

In [ ]:
china.open_tif(modis_china_path, show=True)

In [ ]:
#china.remove_layer('modis')
china.add_layer('MODIS_China.tif', layer_name = 'modis', trim_raster = True)

In [ ]:
show(china.layers['modis'])

We can also **arbitrary** crop a raster/layer: method `crop_layer` can take either starting indices of top/left, ending indices of right/bottom, or coordinates values in lat/long to trim the raster.

We also have a class method `crop_raster` similar to `crop_layer` but we can have any raster as input, which indicates that users do not necessarily need to add a raster as a layer to call that method. (Similar method: `trim_layer`/`trim_raster`, `binarize_layer`/`binarize_raster`)

In [ ]:
china.crop_layer('modis', bounds = (73, 17, 135, 54))
show(china.layers['modis'])

### Categorical Value Extraction, if necessary

In [ ]:
values = np.arange(6, 18)
china.layers['modis_forest'] = geodata.Mask.binarize_raster(china.layers['modis'], values = values)
china.remove_layer('modis')

## MERGING FLATTENING LAYERS

check all the layers

In [ ]:
geodata.Mask.show_all(china.layers)

#### (binary) AND method

In [ ]:
china.merge_layer(attribute_save = False, layers = ['bins', 'forest'])

#### sum method

In [ ]:
china.merge_layer(method = 'sum')

bins: 5%, forest: 25%, slope 40%, and modis_forest 30%. This distribution is completely arbitrary for the purpose of demonstration of the module

In [ ]:
china.merge_layer(method = 'sum', weights = {
    'bins': 0.05, 'forest': 0.25, 'slope': 0.4, 'modis_forest': 0.3
}, trim = True)

## LOADING SHAPES, EXTRACTING SHAPE FROM MASK

In [ ]:
prov_path = shpreader.natural_earth(resolution='10m', category='cultural', name = 'admin_1_states_provinces')
prov_path

In [ ]:
#Check attributes in the shapes contained in path prov_path
#geodata.Mask.shape_attribute(prov_path)

In [ ]:
china_all_shapes = geodata.Mask.get_shape(prov_path, key = 'name_en', 
                         condition_key = 'admin', condition_value = 'China')
china_all_shapes

OR, We can also ignore condition, just take three provinces of China by naming them out

In [ ]:
china_shapes = geodata.Mask.get_shape(prov_path, key = 'name_en', 
                         targets = ['Jiangsu', 'Zhejiang', 'Shanghai'],
                         return_dict = True)
china_shapes

Extract the shapes from the merged_mask:

In [ ]:
china.extract_shapes(china_shapes, crop = True)

In [ ]:
geodata.Mask.show_all(china.shape_mask)

In [ ]:
china

## SHAPE AS LAYER

This is different from shape extractions, as we will simply treat one shp file as a layer, instead of grabbing the merged mask within that shape.

The `add_shape_layer` method take in a dictionary of shapes, a resolution of the result raster with that shape.

In [ ]:
china.add_shape_layer(china_shapes, 
                      reference_layer = 'bins')
show(china.layers['Jiangsu'])

## SAVING MASK

In [ ]:
china.save_mask()

Note that since "Mask has been saved", we can now load the layers or shapes with xarray.

In [ ]:
shape_xr_lst = china.load_shape_xr()
shape_xr_lst['Zhejiang']

In [ ]:
shape_xr_lst['Zhejiang'].plot()

### Load a previously saved Mask



In [ ]:
china_2 = geodata.Mask.load_mask("china")

In [ ]:
china_2

========= End of the mask module functionality demo =========

Note: If you create another object `china_2` that opens the raster `china` is accessing, and then try to save the original `china` without using `china_2.close_files()`, you should expect an error because Python does not want you to rewrite a file that is used by another program. Therefore, `china_2.close_files()` make sures that only `china` mask is having access to the files. `close_files()` will close all the layers in china_2 and make that mask object un-savable. Therefore, it is best to avoid having multiple mask objects accessing the same files.